In [1]:
]activate .

  Activating project at `~/Amazon WorkDocs Drive/My Documents/P2/QuantumOptics.jl/example`


In [2]:
]st

Status `~/Amazon WorkDocs Drive/My Documents/P2/QuantumOptics.jl/example/Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.2
  [f6369f11] ForwardDiff v0.10.34
  [429524aa] Optim v1.7.4
  [21216c6a] Preferences v1.3.0
  [6e0679c1] QuantumOptics v1.0.8 `..#promote-state-and-time`
  [cf7118a7] UUIDs


In [3]:
#import Revise
using BenchmarkTools
@time using QuantumOptics
@time import Preferences, UUIDs
@time Preferences.set_preferences!(UUIDs.UUID("f6369f11-7733-5829-9624-2563aa707210"), "nansafe_mode" => true)
@time import ForwardDiff as FD
@time import Optim

 13.881285 seconds (28.65 M allocations: 1.934 GiB, 7.34% gc time, 8.50% compilation time: 60% of which was recompilation)
  0.022647 seconds (21.63 k allocations: 1.118 MiB, 95.07% compilation time: 100% of which was recompilation)
  0.057959 seconds (72.50 k allocations: 3.817 MiB, 79.18% compilation time: 8% of which was recompilation)
  0.000330 seconds (162 allocations: 12.703 KiB)
  0.000245 seconds (162 allocations: 12.695 KiB)


# System

In [4]:
# 3 level kerr transmon with drive
ba = FockBasis(2)
ω0 = 5.0
α = -0.2
T2 = 1e4
function get_Ht(p::Vector{<:Tp}) where Tp
    A, freq, ϕ, T = p
    op = 2π*([number(ba), 2\create(ba)*number(ba)*destroy(ba), im*(create(ba)-destroy(ba))])
    fun = [t->Tp(ω0), t->Tp(α), t->A*cospi(2t*freq + 2ϕ)*sinpi(t/T)^2]
    H_at_t = LazySum([f(zero(eltype(p))) for f=fun], op)
    function Ht(t,u)
        for k=1:length(fun)
            H_at_t.factors[k] = fun[k](t)
        end
        H_at_t
    end
    return Ht
end

get_Ht (generic function with 1 method)

In [5]:
ψ0 = Operator(SpinBasis(1/2), basisstate(ba, 1), basisstate(ba, 2))
target = ψ0*exp(im*0.5π*dense(sigmax(SpinBasis(1/2)))) # x gate
function cost(par)
    T = par[4]
    Ht = get_Ht(par)
    ts = (0.0, T)
    _, ψT = timeevolution.schroedinger_dynamic(ts, ψ0, Ht)
    1-abs2(tr(target'last(ψT))/2)*exp(-T/T2)
end

cost (generic function with 1 method)

In [6]:
p0 = [0.03, 5.0, 0.25, 100.0]
@btime cost(p0)

  59.758 ms (527965 allocations: 8.08 MiB)


0.010761476896499644

In [7]:
@btime FD.gradient(cost, p0)

  168.576 ms (791799 allocations: 36.27 MiB)


4-element Vector{Float64}:
 -2.183297810465961
 -3.7994486402775753
 -2.3044408856563905e-5
 -0.0006174845877517085

In [8]:
function fg!(L,G,p)
    if G != nothing
        G .= FD.gradient(cost, p)
        any(isnan.(G)) && error("NaN !!")
    end
    cost(p)
end

fg! (generic function with 1 method)

In [9]:
Optim.optimize(Optim.only_fg!(fg!), p0, Optim.Options(show_trace=true))

Iter     Function value   Gradient norm 
     0     1.076148e-02     3.799449e+00
 * time: 0.014918088912963867
     1     1.070336e-02     1.617190e+00
 * time: 5.7250449657440186
     2     1.067363e-02     2.204175e-01
 * time: 6.458923101425171
     3     1.067220e-02     5.033757e-01
 * time: 7.199257135391235
     4     1.005862e-02     1.167300e-01
 * time: 8.880182981491089
     5     1.005848e-02     1.478089e-03
 * time: 9.604084014892578
     6     1.005848e-02     3.408468e-05
 * time: 10.07511305809021
     7     1.005848e-02     3.676830e-03
 * time: 11.781563997268677
     8     9.785697e-03     1.355507e+00
 * time: 14.621310949325562
     9     9.681595e-03     2.427502e+00
 * time: 15.51258897781372
    10     9.582853e-03     3.287200e+00
 * time: 15.977772951126099
    11     9.566007e-03     3.439087e+00
 * time: 16.382487058639526
    12     9.442886e-03     2.313226e+00
 * time: 17.49595618247986
    13     9.024044e-03     1.487944e-01
 * time: 18.10581612586975

 * Status: success (objective increased between iterations)

 * Candidate solution
    Final objective value:     2.286100e-03

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 7.33e-08 ≰ 0.0e+00
    |x - x'|/|x'|          = 3.73e-10 ≰ 0.0e+00
    |f(x) - f(x')|         = 6.38e-14 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 2.79e-11 ≰ 0.0e+00
    |g(x)|                 = 1.44e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   49  (vs limit Inf)
    Iterations:    121
    f(x) calls:    367
    ∇f(x) calls:   367
